# Fair Random Forest

## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import time
import lime
import shap

import warnings
warnings.filterwarnings("ignore")

In [2]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

In [3]:
outputVar = 'Risk'

# this set will be used for training the model, as well as evaluation set for cross-validation
X_train = train.drop(columns= [outputVar],axis=1)
y_train = train[outputVar]

# this set will be used for testing final model performance 
X_test = test.drop(columns= [outputVar],axis=1)
y_test = test[outputVar]

X_train = X_train.drop(columns = ["Checking account_Missing", "Saving accounts_Missing"])
X_test = X_test.drop(columns = ["Checking account_Missing", "Saving accounts_Missing"])

## Training our demonstration classifier

Over the course of this notebook, we will be using this random forest classifier to demonstrate different measures of fairness that we define.

In [4]:
from sklearn.ensemble import RandomForestClassifier

tempForest = RandomForestClassifier(random_state = 0)
_ = tempForest.fit(X_train, y_train)

## Finding Risk Percentages for Males and Females using the Trained Classifier

Given model _myModel_ and dataframe _df_, this function finds the percentages of bad customers for males and females in _df_ according to _myModel_. These percentages will be referred to as $Risk_{male}$ and $Risk_{female}$, respectively.

The function also returns _genderGap_, which is calulated by the formula $|Risk_{male} - Risk_{female}|$.

In [5]:
def getGenderGap(df, myModel):
    males = df.loc[df['Sex_female'] == 0]
    females = df.loc[df['Sex_female'] == 1]
    
    X_males = males.drop(columns=[outputVar],axis=1)
    y_males = males[outputVar]

    X_females = females.drop(columns=[outputVar],axis=1)
    y_females = females[outputVar]
    
    malePredictions = myModel.predict(X_males)
    femalePredictions = myModel.predict(X_females)
    
    maleRiskPercent = np.count_nonzero(malePredictions == True)/malePredictions.shape[0]
    femaleRiskPercent = np.count_nonzero(femalePredictions == True)/femalePredictions.shape[0]
    
    genderGap = abs(maleRiskPercent - femaleRiskPercent)
    return maleRiskPercent, femaleRiskPercent, genderGap

Testing this function on the Random Forest Classifier (Training and Test set):

In [6]:
maleRiskPercent, femaleRiskPercent, genderGap = getGenderGap(train, tempForest)
maleRiskPercent, femaleRiskPercent, genderGap

(0.28113879003558717, 0.35294117647058826, 0.07180238643500109)

In [7]:
maleRiskPercent, femaleRiskPercent, genderGap = getGenderGap(test, tempForest)
maleRiskPercent, femaleRiskPercent, genderGap

(0.1484375, 0.19444444444444445, 0.04600694444444445)

## Finding Risk Percentages for People with different Housing Situations using the Trained Classifier

Given model _myModel_ and dataframe _df_, this function finds the percentages of bad customers for people with different housing situations in _df_ according to _myModel_. These percentages will be referred to as $Risk_{own}$, $Risk_{rent}$ and $Risk_{free}$, respectively.

The function also returns _housingGap_, which is calculated by $|Risk_{own} - Risk_{rent}| + |Risk_{own} - Risk_{free}| + |Risk_{rent} - Risk_{free}|$.

In [8]:
def getHousingGap(df, myModel):
    own = df.loc[df['Housing_own'] == 1]
    rent = df.loc[df['Housing_rent'] == 1]
    free = df.loc[df['Housing_free'] == 1]
    
    X_own = own.drop(columns=[outputVar],axis=1)
    y_own = own[outputVar]
    
    X_rent = rent.drop(columns=[outputVar],axis=1)
    y_rent = rent[outputVar]
    
    X_free = free.drop(columns=[outputVar],axis=1)
    y_free = free[outputVar]
    
    ownPredictions = myModel.predict(X_own)
    rentPredictions = myModel.predict(X_rent)
    freePredictions = myModel.predict(X_free)
    
    ownRiskPercent = np.count_nonzero(ownPredictions == True)/ownPredictions.shape[0]
    rentRiskPercent = np.count_nonzero(rentPredictions == True)/rentPredictions.shape[0]
    freeRiskPercent = np.count_nonzero(freePredictions == True)/freePredictions.shape[0]
    
    housingGap = abs(ownRiskPercent - rentRiskPercent)
    + abs(ownRiskPercent - freeRiskPercent)
    + abs(rentRiskPercent - freeRiskPercent)
    return ownRiskPercent, rentRiskPercent, freeRiskPercent, housingGap

Testing this function on the Random Forest Classifier (Training and Test set):

In [9]:
ownRiskPercent, rentRiskPercent, freeRiskPercent, housingGap = getHousingGap(train, tempForest)
ownRiskPercent, rentRiskPercent, freeRiskPercent, housingGap

(0.26223776223776224,
 0.4125874125874126,
 0.38823529411764707,
 0.15034965034965037)

In [10]:
ownRiskPercent, rentRiskPercent, freeRiskPercent, housingGap = getHousingGap(test, tempForest)
ownRiskPercent, rentRiskPercent, freeRiskPercent, housingGap

(0.11347517730496454,
 0.3888888888888889,
 0.13043478260869565,
 0.27541371158392436)

## Finding Inequality using Cross-Validation

As defined in this project, $inequality = |Risk_{male} - Risk_{female}| + |Risk_{own} - Risk_{rent}| + |Risk_{own} - Risk_{free}| + |Risk_{rent} - Risk_{free}|$. ($inequality$ is therefore equal to the sum of _genderGap_ and _housingGap_.)

To ensure the accuracy of  the $inequality$ metric, we use a cross-validation approach. Here, we use 10 folds, where for each fold, $inequality$ is measured when the model is trained on the rest of the train dataset. Then, the mean of these $inequality$ values that were sampled is used to return the final result.

In [11]:
from sklearn.model_selection import StratifiedKFold

n_splits = 10

def getInequalityCV(myModel):
    
    inequalities = np.array([])
    
    kf = StratifiedKFold(n_splits = n_splits, shuffle = True, random_state = 0)
    
    mySplits = kf.split(X_train, y_train)
    for trainFold, evalFold in mySplits:
        X_train_temp = X_train.loc[trainFold]
        y_train_temp = y_train.loc[trainFold]

        evalTemp = train.loc[evalFold]

        myModel.fit(X_train_temp, y_train_temp)

        maleRiskPercent, femaleRiskPercent, genderGap = getGenderGap(evalTemp, myModel)
        ownRiskPercent, rentRiskPercent, freeRiskPercent, housingGap = getHousingGap(evalTemp, myModel)
        
        inequality = genderGap + housingGap
        
        inequalities = np.append(inequalities, inequality)
        
    return inequalities

Testing this function on the random forest classifier:

In [12]:
getInequalityCV(tempForest).mean()

0.26499386043387646

## Setting up the Optimization Problem

To evaluate the performance of the model, we use the function $f(p) = -auc\_score + c * inequality$.

This function balances the maximization of $auc\_score$ and the minimization of $inequality$ using weight $c$. As model evaluation metric, $auc\_score$ was chosen as it balances false positives and false negatives. Again, we use cross-validation to ensure reliability of the $auc\_score$ metric.

In [13]:
from sklearn.model_selection import cross_val_score

myRandom = 0
c = 1

def objective(trial):
    tempForest = RandomForestClassifier(random_state = 0)
    
    global myRandom
    myCV = StratifiedKFold(n_splits=10, shuffle=True, random_state=myRandom)
    auc = cross_val_score(tempForest, X_train, y_train, scoring = 'roc_auc', cv = myCV).mean()
    inequality = getInequalityCV(tempForest).mean()
    myRandom = myRandom + 1
    
    result = -auc + c * inequality
    return auc, inequality, result

This is our $auc\_score$, $inequality$, and $f(p)$ value for the random forest classifier.

In [14]:
objective(None)

(0.6769295725108225, 0.26499386043387646, -0.41193571207694607)

# TODO

- Check metrics before & after optimization of $c$
  - Accuracy (ROC_AUC)
  - Inequality
  - Risk Percentages for different subgroups
  - Graph with c on x axis, inequality and accuracy on y axis
  - Graph with c on x axis, and risk percentages on y axis
  
- Analyse model before & after optimization of $c$ using LIME & SHAP
- Use other models (logistic regression, decision tree, bayesian classifier) and see how they perform regarding metrics (only if time allows)